In [ ]:
print("Gashaw")import pandas as pd
import numpy as np

# Load datasets
fraud_data = pd.read_csv('../data/Fraud_Data.csv')
ip_to_country = pd.read_csv('../data/IpAddress_to_Country.csv')
creditcard_data = pd.read_csv('../data/creditcard.csv')

# Handle missing values
fraud_data.dropna(inplace=True)
creditcard_data.dropna(inplace=True)

# Data cleaning
fraud_data.drop_duplicates(inplace=True)
creditcard_data.drop_duplicates(inplace=True)

# Convert IP addresses to integer format
def ip_to_int(ip):
    return int(''.join(f'{int(octet):08b}' for octet in ip.split('.')), 2)

fraud_data['ip_int'] = fraud_data['ip_address'].apply(ip_to_int)

# Merge datasets for geolocation analysis
ip_to_country['lower_bound_ip_address'] = ip_to_country['lower_bound_ip_address'].apply(ip_to_int)
ip_to_country['upper_bound_ip_address'] = ip_to_country['upper_bound_ip_address'].apply(ip_to_int)

fraud_data = pd.merge_asof(fraud_data.sort_values('ip_int'), 
                           ip_to_country.sort_values('lower_bound_ip_address'), 
                           left_on='ip_int', 
                           right_on='lower_bound_ip_address')

# Feature engineering
fraud_data['hour_of_day'] = pd.to_datetime(fraud_data['purchase_time']).dt.hour
fraud_data['day_of_week'] = pd.to_datetime(fraud_data['purchase_time']).dt.dayofweek

# Normalization and scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
creditcard_data['Amount'] = scaler.fit_transform(creditcard_data[['Amount']])

# Encode categorical features
fraud_data = pd.get_dummies(fraud_data, columns=['source', 'browser', 'sex'], drop_first=True)

# Save preprocessed data
fraud_data.to_csv('../data/processed_fraud_data.csv', index=False)
creditcard_data.to_csv('../data/processed_creditcard_data.csv', index=False)